In [18]:
import requests
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import json

In [37]:
# inputs
codeList = ["AAPL"]
benchmark = "VOO"
start_date = "2016-03-13"
end_date = "2020-03-13"
regressionFactors = ["MktMinRF", "SMB" ]
with open('../data/token.txt', 'r') as file:
    token = file.read()

In [38]:
if benchmark != "None":
    codeList.append(benchmark)

In [39]:
def iexHistoricalPriceRequest (codeList,token):
    indexData = pd.DataFrame()
    codeListString = ','.join(codeList)
    iexRequest = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={codeListString}&types=chart&range=max&token={token}&filter=date,close'
    iexResponse = requests.get(iexRequest).json()
    for x in codeList:
        singleCode = pd.DataFrame(iexResponse[x]['chart'])
        singleCode.columns = ['date',x]
        if x == codeList[0]:
            indexData = indexData.append(singleCode)
        else:
            indexData = pd.merge(indexData,singleCode)
    indexData = indexData.sort_values(by='date').reset_index(drop=True)
    return(indexData)

In [40]:
indexData = iexHistoricalPriceRequest(codeList,token)
indexData=indexData.rename(columns = {indexData.columns[-1] :'benchmark'})
indexData = indexData[indexData['date']>=start_date]
indexData = indexData[indexData['date']<=end_date]
indexData = indexData.reset_index(drop=True)

In [41]:
indexData['portfolioreturn'] = (indexData[codeList[0]] - indexData[codeList[0]].shift(1))/indexData[codeList[0]].shift(1)
indexData['benchmarkreturn'] = (indexData.benchmark - indexData.benchmark.shift(1))/indexData.benchmark.shift(1)

In [42]:
frenchfama = pd.read_csv('data/ff5factordaily.CSV')
frenchfama = frenchfama[frenchfama['date']>start_date]
frenchfama = frenchfama[frenchfama['date']<=end_date].reset_index(drop=True)
regression_data = pd.concat([indexData,frenchfama],axis = 1,join = 'inner')

In [44]:
regressionFactors = ' + '.join(regressionFactors)

In [45]:
model = smf.ols(formula=f'portfolioreturn ~ {regressionFactors}', data=regression_data)
results = model.fit()

In [48]:
def result_summary_to_dataframe(results):
    '''take the result of an statsmodel results table and transforms it into a dataframe'''
    #https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.RegressionResults.html link if interested in adding more results
    summary_json = {
    'nobs': results.nobs,
    'rsquared': results.rsquared,
    'rsquared_adj': results.rsquared_adj,
    'fvalue': results.fvalue
    }
    return summary_json

In [49]:
def result_pvalues_to_dataframe(results):
    '''take the result of an statsmodel results table and transforms it into a dataframe'''
    pvals = results.pvalues
    coeff = results.params
    conf_lower = results.conf_int()[0]
    conf_higher = results.conf_int()[1]

    results_df = pd.DataFrame({"pvals":pvals,
                               "coeff":coeff,
                               "conf_lower":conf_lower,
                               "conf_higher":conf_higher
                                })

    #Reordering...
    results_df = results_df[["coeff","pvals","conf_lower","conf_higher"]]
    return results_df

In [95]:
pvaluesJSON = result_pvalues_to_dataframe(results).to_json()
summaryJSON = result_summary_to_dataframe(results)
pvaluesJSON = json.loads(pvaluesJSON)
outputJSON = summaryJSON
outputJSON.update(pvaluesJSON)

In [50]:
result_pvalues_to_dataframe(results)

,coeff,pvals,conf_lower,conf_higher
Intercept,0.000821,2.485656e-01,-0.000575,0.002218
MktMinRF,0.012128,1.523823e-57,0.010733,0.013522
SMB,-0.003702,7.412899e-03,-0.006409,-0.000995
